# AMSC698K Homework 11
##### Elijah Kin

In [1]:
# TODO

### Problem A
Notebook `QHED_intro.ipynb` introduces a quantum algorithm to extract color boundaries (edges) in images. The 512x512-pixel gray-tone picture at the end (`qhed_image.jpg`) features strong contours, which are easy to identify.

1. Identify the edges in this picture using a classical and a quantum technique. If your laptop doesn't enough RAM, you might need to divide the image into 4 sub-images, analyze each of them, and add the results to yield the edge image.

In [2]:
# TODO

2. Run your quantum circuit on "ionq_simulator" with noise-model "forte-1", adjust thresholds (tolerances) if necessary, and submit the code to "ionq_qpu.forte-1".

In [3]:
# TODO

### Problem B
1. Notebook `FER_intro.ipynb` contains a simple code to perform an analysis of facial expressions. The routine "faces_classifier" simply compares a test image with one random example of the "happy face" subset and one of the "sad face" subset. Find more reliable ways to classify a random test image. (You may modify the quantum circuit if desired.)

In [4]:
# TODO

2. Explain how the quantum circuit (pictured in `FER_intro.ipynb`) performs the classification starting from the state $|0\rangle_{aux} |0\rangle_{index} |00\dots0\rangle_{data} |0\rangle_{class}$.

TODO

3. (extra credit) Instead of using the small subset of "happy" and "sad" faces presented in class (file: ffhq_subset.zip Download ffhq_subset.zip) find a way to classify reliably the larger set of FFHQ images and landmark points in Files -> Course Material -> Homework.

In [5]:
# TODO